# Classe ECDH

In [8]:
class ECDH:
    def __init__(self, n):
        self.n = n
        self.p = 2
        self.q = self.p^n
        self.K = GF(self.q)
        self.b, self.G = self.setup()

    def setup(self):
        isGenerator = False
        while true:
            b = self.K(ZZ.random_element(1, self.q).digits(base=self.p))

            E = EllipticCurve(self.K, [1,1,0,0,b])
            order = E.order()
            prime_factors = [ f for (f,_) in factor(order) ]
            highest_prime_factor = prime_factors[-1]

            if highest_prime_factor >= 2^(self.n - 1):

                for i in range(0, 1000):
                    Q = E.random_element()

                    for pf in prime_factors:
                        if pf * Q == E(0):
                            break
                    if pf == highest_prime_factor:
                        isGenerator = True
                        break
                if isGenerator:
                    G = (order//highest_prime_factor) * Q
                    break
        return b, G

    def gen_key_pair(self):
        N = self.G.order()
        private_key = ZZ.random_element(1, N)
        public_key = private_key * self.G
        return private_key, public_key
    
    def gen_shared_key(self, private_key, peer_public_key):
        return private_key * peer_public_key

# Setup

In [9]:
ecdh = ECDH(163)

# Geração do segredo partilhado

In [10]:
A_private_key, A_public_key = ecdh.gen_key_pair()
B_private_key, B_public_key = ecdh.gen_key_pair()

A_shared_key = ecdh.gen_shared_key(A_private_key, B_public_key)
B_shared_key = ecdh.gen_shared_key(B_private_key, A_public_key)

print A_shared_key == B_shared_key

True
